In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import matplotlib.pyplot as plt
%matplotlib inline 

# Generate dummy data
import numpy as np

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [4]:
# https://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
# tensorboard
# https://www.youtube.com/watch?v=lV09_8432VA
# https://www.youtube.com/watch?v=BqgTU7_cBnk - stydy model

import os
import tensorflow as tf
from keras.callbacks import TensorBoard

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [12]:

# Tensorboard
tbCallBack = keras.callbacks.TensorBoard(log_dir='/Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()

tbCallBack.set_model(model)

# https://stackoverflow.com/questions/47877475/keras-tensorboard-plot-train-and-validation-scalars-in-a-same-figure/48393723

model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

weights, biases = model.layers[0].get_weights()
#tf.summary.histogram("weights", W1)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          epochs=50,
          batch_size=128, callbacks=[TrainValTensorBoard(write_graph=False)])

#plt.plot(history)

score = model.evaluate(x_test, y_test, batch_size=128)

# export model
# https://keras.io/models/about-keras-models/
print model.summary()
#print model.get_weights()


Epoch 1/50
1000/1000 [==============================] - 0s 328us/step - loss: 0.7153 - acc: 0.4880
Epoch 2/50
1000/1000 [==============================] - 0s 38us/step - loss: 0.7036 - acc: 0.5050
Epoch 3/50
1000/1000 [==============================] - 0s 44us/step - loss: 0.7027 - acc: 0.5050
Epoch 4/50
1000/1000 [==============================] - 0s 36us/step - loss: 0.6938 - acc: 0.5090
Epoch 5/50
1000/1000 [==============================] - 0s 32us/step - loss: 0.6968 - acc: 0.4840
Epoch 6/50
1000/1000 [==============================] - 0s 49us/step - loss: 0.6972 - acc: 0.4990
Epoch 7/50
1000/1000 [==============================] - 0s 51us/step - loss: 0.6997 - acc: 0.5060
Epoch 8/50
1000/1000 [==============================] - 0s 42us/step - loss: 0.6940 - acc: 0.5050
Epoch 9/50
1000/1000 [==============================] - 0s 38us/step - loss: 0.6948 - acc: 0.5030
Epoch 10/50
1000/1000 [==============================] - 0s 41us/step - loss: 0.6950 - acc: 0.5430
Epoch 11/50
1000/1

In [ ]:
# perf training
# Training Performance: A user’s guide to converge faster (TensorFlow Dev Summit 2018)

# https://www.youtube.com/watch?v=eBbEDRsCmv4 - Hands-on TensorBoard (TensorFlow Dev Summit 2017)